# Contents

[**1. Connect to Reddit and choose subreddits to download**](#reddit_connect)

[**2. Build the SQL database**](#build_sql)

[**3. Pull all the subreddit data and save it to SQL DB**](#pull_reddit)

[**4. Populate Slack channels with Reddit data**](#populate_slack)

[**5. Commit & close connection**](#close)

---

In [1]:
import praw, psycopg2, sys, time, subprocess, random, os
import pandas as pd
from slackclient import SlackClient
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT # Trick to be able to create a DB

---

# <a name="reddit_connect">1. Connect to Reddit and choose subreddits to download</a>

Open my Postgres app

In [2]:
# subprocess.call(["/usr/bin/open", "-n", "-a", "/Applications/Postgres.app"]);

Connect to Reddit API via `PRAW`

In [3]:
reddit = praw.Reddit(client_id = os.environ["PRAW_CLIENT_ID"],
                     client_secret = os.environ["PRAW_CLIENT_SECRET"],
                     password = os.environ["PRAW_PASSWORD"],
                     user_agent = 'slackbot by /u/aficnar',
                     username = 'aficnar')

Choose subreddits to download:

In [4]:
subreddit_try = 'MachineLearning'
subreddit = reddit.subreddit(subreddit_try)
print(subreddit.title)

Machine Learning


In [9]:
chosen_subreddits = []

---

# <a name="build_sql">2. Build the SQL database</a>

Create a local SQL database:

In [10]:
# con = psycopg2.connect(database = 'postgres', user = 'aficnar')
# con.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
# cur = con.cursor()
# cur.execute("CREATE DATABASE slack_police")

Connect to the DB and, if necessary, drop the previous tables (from failed attempts):

In [11]:
con = psycopg2.connect(database = 'slack_police', user = 'aficnar')
cur = con.cursor()
#cur.execute("DROP TABLE main_subreddits")
#cur.execute("DROP TABLE main_submissions")
#cur.execute("DROP TABLE main_comments")

Create a table that will store the subreddit names, id's, etc.

In [12]:
#cur.execute("""
#            CREATE TABLE main_subreddits
#            (
#            id VARCHAR(255) NOT NULL,
#            name TEXT NOT NULL,
#            title TEXT NOT NULL, 
#            created INT NOT NULL,
#            total_submissions INT NOT NULL
#            )
#            """)

Put the chosen subreddits in it:

In [13]:
for i in range(len(chosen_subreddits)):
    subreddit = reddit.subreddit(chosen_subreddits[i])
    db_tuple = (subreddit.id, chosen_subreddits[i], subreddit.title, 
                int(subreddit.created), 0)
    cur.execute("INSERT INTO main_subreddits VALUES (%s, %s, %s, %s, %s)", db_tuple)

In [14]:
pd.read_sql("SELECT * FROM main_subreddits", con)

,id,name,title,created,total_submissions
0,2qr9w,diving,Scuba Diving and Snorkeling,1231768360,1149
1,2rr72,Handball,Handball,1273801402,793
2,2rdw8,corgi,Corgi Subreddit,1260187617,0
3,2sptq,datascience,Data Science,1312659822,0
4,2r3gv,MachineLearning,Machine Learning,1248906884,0


Create tables that will contain the submissions and comments:

In [15]:
#cur.execute("""
#            CREATE TABLE main_submissions
#            (
#            id VARCHAR(255) NOT NULL,
#            subreddit_id VARCHAR(255) NOT NULL,
#            content TEXT
#            )
#            """)

In [16]:
#cur.execute("""
#            CREATE TABLE main_comments
#            (
#            id VARCHAR(255) NOT NULL,
#            subreddit_id VARCHAR(255) NOT NULL,
#            submission_id VARCHAR(255) NOT NULL,
#            content TEXT
#            )
#            """)

Get all the relevant tables and columns in our DB and display them nicely:

In [17]:
sql_query = """
            SELECT table_name 
            FROM information_schema.tables
            WHERE table_name LIKE 'main_%'
            """
table_list = pd.read_sql(sql_query, con)

In [18]:
sql_query = """
            SELECT column_name, table_name 
            FROM information_schema.columns
            WHERE table_name LIKE 'main_%'
            """
col_list = pd.read_sql(sql_query, con)

In [19]:
# Generate a dictionary containing all column names
DF_dict = {}
for name in list(table_list['table_name']):
    DF_dict[name] = list(col_list[col_list['table_name'] == name]['column_name'])

# Fill it up so all lists are equal length
max_length = max([len(f) for f in DF_dict.values()])
for key, value in DF_dict.items():
    DF_dict[key] = DF_dict[key] + ['-----'] * (max_length - len(DF_dict[key]))

# Create a DataFrame
all_columns = pd.DataFrame(DF_dict)
all_columns

,main_comments,main_submissions,main_subreddits
0,id,id,id
1,subreddit_id,subreddit_id,name
2,submission_id,content,title
3,content,-----,created
4,-----,-----,total_submissions


---

# <a name="pull_reddit">3. Pull all the subreddit data and save it to SQL DB</a>

First, couple of functions that we'll use to display the remaining time:

In [20]:
def stringify(num):
    num_string = str(num)
    if len(num_string) == 1: num_string = '0' + num_string
    return num_string

def convert_secs(secs_float):
    secs = int(round(secs_float))
    sec_display = secs % 60
    mins = secs // 60
    mins_display = mins % 60
    hrs_display = mins // 60
    export_string = stringify(hrs_display) + ':' + stringify(mins_display) + ':' + stringify(sec_display)
    return export_string

Put the subreddits to be downloaded in `pull_df` (useful for adding other subreddits later on)

In [21]:
all_df = pd.read_sql("SELECT * FROM main_subreddits", con)
pull_df = all_df.iloc[2:]
pull_df.index = range(pull_df.shape[0])
pull_df

,id,name,title,created,total_submissions
0,2rdw8,corgi,Corgi Subreddit,1260187617,0
1,2sptq,datascience,Data Science,1312659822,0
2,2r3gv,MachineLearning,Machine Learning,1248906884,0


Loop over the above subreddits, submissions and comments and save them to DB -- and if you've crossed the `comments_threshold`, stop early

In [62]:
comments_threshold = 5000

In [63]:
for i in range(pull_df.shape[0]):
    t_start = time.time()
    print "Subreddit " + str(i + 1) + " / " + str(pull_df.shape[0]) + ": " + pull_df['title'][i]
    # Get relevant subreddit info
    subreddit = reddit.subreddit(pull_df['name'][i])
    subreddit_id = pull_df['id'][i]
    comment_count = 0
    # Loop over all the submissions
    for submission in subreddit.submissions():
        submission.comments.replace_more(limit = 0); # Important for deep comments
        submission_tuple = (submission.id, subreddit_id, submission.selftext)
        comment_tuples = []
        # Loop over all the comments in this submission
        comment_list = submission.comments.list()
        for comment in comment_list:
            comment_tuples.append((comment.id , subreddit_id, submission.id, comment.body))
        if len(comment_tuples) > 0:
            comment_str = ','.join(cur.mogrify("(%s,%s,%s,%s)", x) for x in comment_tuples)
            cur.execute("INSERT INTO main_comments VALUES " + comment_str) 
        cur.execute("INSERT INTO main_submissions VALUES (%s, %s, %s)", submission_tuple)
        comment_count = comment_count + len(comment_list)
        # The rest is for pretty tracking of progress
        sys.stdout.write("\rComments processed: %d" % comment_count)
        sys.stdout.flush()
        # Exit if you've downloaded more than comments_threshold comments
        if comment_count > comments_threshold: break
    t_finish = time.time()
    print
    print 'Runtime: ' + convert_secs(t_finish - t_start)
    print("-" * 50)

Subreddit 1 / 3: Corgi Subreddit
Comments processed: 5006 / 
Runtime: 00:20:53
--------------------------------------------------
Subreddit 2 / 3: Data Science
Comments processed: 5008 / 
Runtime: 00:16:18
--------------------------------------------------
Subreddit 3 / 3: Machine Learning
Comments processed: 5013 / 
Runtime: 00:07:51
--------------------------------------------------


Check the total amount of submissions and comments downloaded:

In [22]:
all_comments = pd.read_sql("SELECT * FROM main_comments", con)
all_submissions = pd.read_sql("SELECT * FROM main_submissions", con)
print all_comments.shape[0]
print all_submissions.shape[0]

26023
4770


---

# <a name="populate_slack">4. Populate Slack channels with Reddit data<a/>

There's a Slack team called `Slack Police`, and I created 5 bots there. 
* The idea is to pick one of the first four bots here randomly as we upload the messages to Slack, so it looks like a real conversation. 
* The last bot, `officer_newman` will be the one monitoring discussions and warning about users going off-topic.

In [5]:
bot_tokens = [os.environ["SLACKBOT_TOKEN_JERRY"],
              os.environ["SLACKBOT_TOKEN_ELAINE"],
              os.environ["SLACKBOT_TOKEN_COSMO"],
              os.environ["SLACKBOT_TOKEN_GEORGE"],
              os.environ["SLACKBOT_TOKEN_NEWMAN"]]
bot_names = ['jerry', 'elaine', 'cosmo', 'george', 'officer_newman']

Initialize the Slack clients via `slackclient` library

In [6]:
slack_uploaders = [SlackClient(bot_tokens[i]) for i in range(4)]

Let's make sure the uploaders see the channels:

In [7]:
channel_list = slack_uploaders[0].api_call("channels.list")['channels']
[(c['id'], c['name']) for c in channel_list]

[(u'C3WJDNPCJ', u'ex1_diving'),
 (u'C3WJDPEHG', u'ex2_handball'),
 (u'C3WV35AEN', u'ex3_corgi'),
 (u'C3W93G2F3', u'ex4_data_science'),
 (u'C3XKZSM1V', u'ex5_machine_learning'),
 (u'C3UTPTUMT', u'general'),
 (u'C3UTDRVT6', u'random')]

Select the subreddits to be uploaded and the corresponding channels:

In [26]:
all_subreddit_upload_names = pd.read_sql("SELECT name FROM main_subreddits", con)['name'].tolist()
all_subreddit_upload_names

['diving', 'Handball', 'corgi', 'datascience', 'MachineLearning']

In [27]:
all_channel_upload_list = [channel_list[i]['id'] for i in range(len(all_subreddit_upload_names))]
all_channel_upload_list

[u'C3WJDNPCJ', u'C3WJDPEHG', u'C3WV35AEN', u'C3W93G2F3', u'C3XKZSM1V']

In [29]:
subreddit_upload_names = all_subreddit_upload_names[2:]
channel_upload_list = all_channel_upload_list[2:]
subreddit_upload_names

['corgi', 'datascience', 'MachineLearning']

Loop over them and, for now, just upload the first `comment_limit` comments, also pausing for 1s, due to Slack's API rate limit:

In [32]:
comment_limit = 1000

In [ ]:
cnt = 0
for i_sub in range(len(subreddit_upload_names)):
    # Get the comments from a given subreddit
    sql_query = """
                SELECT content 
                FROM main_comments 
                WHERE subreddit_id = (
                    SELECT id 
                    FROM main_subreddits 
                    WHERE name = '%s'
                    )
                """ % subreddit_upload_names[i_sub]
    comments = pd.read_sql(sql_query, con)['content'].tolist()[:comment_limit]
    # Upload each comment to Slack as a random bot
    for c in comments:
        bot_index = random.randint(0,3)
        slack_uploaders[bot_index].api_call("chat.postMessage", channel = channel_upload_list[i_sub],
                                            text = c, as_user = True);
        time.sleep(1)
        cnt = cnt + 1
        output_string = "\rTime remaining: " + convert_secs(comment_limit * len(subreddit_upload_names) - cnt)
        sys.stdout.write(output_string)
        sys.stdout.flush()

---

# <a name="close">5. Commit & close connection</a>

In [98]:
con.commit()
cur.close()
con.close()